In [7]:
#installations
pip install selenium 

SyntaxError: invalid syntax (<ipython-input-7-df8fc83cadcc>, line 2)

In [2]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
import io
import requests
import PIL.Image as Image
import hashlib

In [2]:
import selenium
from selenium import webdriver

DRIVER_PATH = '/home/naitik_1502/Scrapping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [22]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=50):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [29]:
search_term = 'island high resolution'

search_and_download(search_term = search_term ,driver_path = DRIVER_PATH )

Found: 100 search results. Extracting links from 0:100
Found: 51 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR-1wIEwSDEVg2HzBRMkA3z7dZ3m69acme2cei2GaIhrDgO_F_s&usqp=CAU - as ./images/island_high_resolution/936e114950.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQvEew0bd6bJPerCdYEJGPqJaGiVUKckvr0YJl1gAMzhDqcipgn&usqp=CAU - as ./images/island_high_resolution/53eac19104.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTRG7rjET24rXemHfakD0GLBdLsd_LXWWUhaJydxRnKs-HgPxCo&usqp=CAU - as ./images/island_high_resolution/d7be796741.jpg
SUCCESS - saved https://i.pinimg.com/originals/0f/f6/12/0ff612687838dec224bc1e2c9108d6f2.jpg - as ./images/island_high_resolution/1d299a9b13.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS9D2_fgiFcrCdfwk3YPqxnp5urcVuFv6kIrF9vGFP9JWcJef_2&usqp=CAU - as ./images/island_high_resolution/d0a1fa9f80.jpg
SUCCESS - saved https://encrypted-t

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTf7FMPLnwIN2DCo2Bknkm1B2Sg4Qb3LejT4Ferm-bEL7tVZnTG&usqp=CAU - as ./images/island_high_resolution/e97ed95dcf.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSfOjXLBun2J_tHc3Zw2nlSCK7Dl6NhjXOCcgtDUDiZC5TD9cgE&usqp=CAU - as ./images/island_high_resolution/d4b05d7923.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTxxOCETmD-ym1rb8y0ujZV0K1ep-CowgFpPR-uPQp7_lXLyXXL&usqp=CAU - as ./images/island_high_resolution/0a8402aca8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ3R-0DbjH85WQUHQDmoc214JdC89FL611i8W72p4L6HRPe4y3X&usqp=CAU - as ./images/island_high_resolution/3c7ec021b8.jpg
SUCCESS - saved https://c4.wallpaperflare.com/wallpaper/860/720/995/5bd34ca06ca9b-wallpaper-preview.jpg - as ./images/island_high_resolution/a486ef2d20.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR85Q8PzH1mItuUbnuArWotNaPwUJ-rAbadNX